In [41]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
import re
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from oauth2client import file, client, tools
import os
import httplib2

In [42]:
# Run a new search, and extract the corresponding search id
search_string = 'https://search.larryhotz.com/search/results/?city=Denver&zip=all&neighborhood=all&type=res&type=con&search_status=SOLD&sold_price_min=all&sold_price_max=all&sold_within=1&short_sale=all&beds_min=all&baths_min=all&area_min=all&amenities=all&lot_size_min=all&year_built_min=all&levels=all&school_district=all&basement=all&sort_latest=true'
search_page = requests.get(search_string)
search_html = soup(search_page.content, 'html.parser')
search_html = search_html.find('a', class_="savesearch_button")['href']
search_code = re.findall("/\w{4}/", search_html)[0]

search_code

'/4b8b/'

In [43]:
def clean_dat_string(value):
    value = value.text
    result = re.sub(r'[\t\r\n]', '', value).strip()
    return result

def parse_prop(prop_details, label):
    value = prop_details.find(text=label)
    if value is not None: 
        value = value.next_element.next_element.text
    return value

In [46]:
# Initiates blank dataframe with the columns I want
result_df = pd.DataFrame(columns=['sa1','sa2','sold_price','closing_date','prop_type','year_built','sq_ft','bedrooms','bathrooms','sub_div','sub_area','year_built','taxes','school_district','garage_spaces','parking_spaces','basement','hoa_fees','financial_terms'])

# Loops through urls, scrapes the data, and saves it to the dataframe
for i in range(0,1000):
    result_url = 'https://search.larryhotz.com/search/details'+search_code+str(i)
    result_page = requests.get(result_url)
    result_page = soup(result_page.content, 'html.parser')
    
    
    if i != 404:
        prop_details = result_page.findAll(class_='property_detail_specs')[0]
        prop_details2 = result_page.findAll(class_='property_detail_specs')[1]
        
        
        
        if parse_prop(prop_details,'Closing Date') ==  'Mar 30, 2021':
            break

        result_df.loc[i,'sa1'] = clean_dat_string(result_page.find('h2'))
        result_df.loc[i,'sa2'] = clean_dat_string(result_page.find('h3'))
        result_df.loc[i,'closing_date'] = parse_prop(prop_details,'Closing Date')
        result_df.loc[i,'sold_price'] = parse_prop(prop_details,'List Price')
        result_df.loc[i,'sold_price'] = parse_prop(prop_details,'Sold Price')
        result_df.loc[i,'prop_type'] = parse_prop(prop_details,'Type')
        result_df.loc[i,'year_built'] = parse_prop(prop_details,'Year Built')
        result_df.loc[i,'sq_ft'] = parse_prop(prop_details,'Total Sqft')
        result_df.loc[i,'bedrooms'] = parse_prop(prop_details,'Bedrooms')
        result_df.loc[i,'bathrooms'] = parse_prop(prop_details,'Bathrooms')
        result_df.loc[i,'sub_div'] = parse_prop(prop_details,'Subdivision')
        result_df.loc[i,'sub_area'] = parse_prop(prop_details,'Sub-Area')
        result_df.loc[i,'year_built'] = parse_prop(prop_details,'Year Built')
        result_df.loc[i,'taxes'] = parse_prop(prop_details2,'Taxes')
        result_df.loc[i,'school_district'] = parse_prop(prop_details2,'School District')
        result_df.loc[i,'garage_spaces'] = parse_prop(prop_details2,'Garage Spaces')
        result_df.loc[i,'parking_spaces'] = parse_prop(prop_details2,'Parking Spaces')
        result_df.loc[i,'basement'] = parse_prop(prop_details2,'Basement')
        result_df.loc[i,'hoa_fees'] = parse_prop(prop_details2,'Total HOA Fees')
        result_df.loc[i,'financial_terms'] = parse_prop(prop_details2,'Financial Terms')
        
        # Redneck progress bar
        print(i)
        
result_df


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,sa1,sa2,sold_price,closing_date,prop_type,year_built,sq_ft,bedrooms,bathrooms,sub_div,sub_area,year_built,taxes,school_district,garage_spaces,parking_spaces,basement,hoa_fees,financial_terms
0,3628 Dexter Court,"Denver, CO 80207","$505,000","Apr 20, 2021",Single Family Residential,2002,1234,None,None,Overlook At Park Hill,Overlook At Park Hill,2002,"$1,877",Denver 1,2,2,None,$45,Conventional
1,9725 E Harvard Avenue Unit 374,"Denver, CO 80231","$107,000","Apr 19, 2021",Condo,1972,721,1,1,Woodstream Falls,Woodstream Falls,1972,$590,Denver 1,None,None,None,$334,Conventional
2,2255 S Monroe Street,"Denver, CO 80210","$2,300,000","Apr 19, 2021",Single Family Residential,1947,1542,3,2,University Park/Observatory Park,University Park/Observatory Park,1947,"$3,664",Denver 1,2,2,None,None,Cash
3,2552 E Alameda Avenue Unit 65,"Denver, CO 80209","$1,250,000","Apr 19, 2021",Condo,1983,3690,3,3,Polo Club North,Polo Club North,1983,"$5,412",Denver 1,2,2,Partial,"$1,249",Conventional
4,3700 S Glencoe Street,"Denver, CO 80237","$800,000","Apr 19, 2021",Single Family Residential,1959,2960,3,2,Cherry Point,Cherry Point,1959,"$2,491",Denver 1,2,2,"Crawl Space, Partial",None,Cash
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2625 Oneida Street,"Denver, CO 80207","$500,000","Mar 31, 2021",Single Family Residential,1948,2426,5,4,Park Hill,Park Hill,1948,"$2,532",Denver 1,1,1,Crawl Space,None,Conventional
766,3463 Lawrence Street,"Denver, CO 80205","$790,000","Mar 31, 2021",Single Family Residential,1998,1152,3,2,Rino,Rino,1998,"$2,305",Denver 1,None,None,Crawl Space,None,Cash
767,891 14th Street Unit 2601,"Denver, CO 80202","$980,000","Mar 31, 2021",Condo,2009,1639,2,3,Downtown,Downtown,2009,"$5,069",Denver 1,2,2,None,$688,Conventional
768,4225 Shangri La Drive,"Denver, CO 80246","$950,000","Mar 31, 2021",Single Family Residential,1978,3220,3,3,Hilltop,Hilltop,1978,"$5,755",Denver 1,2,2,Crawl Space,None,Cash


In [49]:
scopes = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
json_key_filename = 'denver-311215-14d0c79b4aa2.json'

credentials = Credentials.from_service_account_file(json_key_filename, scopes=scopes)
service = build('sheets', 'v4', credentials=credentials)
gc = gspread.authorize(credentials)


In [51]:
client_email = 'create-sales@denver-311215.iam.gserviceaccount.com'
spreadsheet_key = '1jRBmZigXTJHO5Y7HoJ6kRnyTSekZ4BS7je1lwfKqpRg'
wks_name = 'Sheet1'

sh1 = gc.open_by_key(spreadsheet_key).get_worksheet(0)
df_test = dataframe = pd.DataFrame(sh1.get_all_records())
df_test

set_with_dataframe(sh1, result_df)